# homeFinder
This notebook describeds my Capstone project : homeFinder. 

## Configuration preparation
Import packages

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium 
import folium # plotting library

# !pip install mpuimport mpu
!pip install mpu --user

import mpu
#!pip install git+https://github.com/MartinThoma/mpu.git
#!conda install -c conda-force mpu=0.23.0 --yes

#print('Folium installed')
print('Libraries imported.')

You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.
Libraries imported.


Set foursquare crendentials

In [3]:
# @hidden_cell
CLIENT_ID = 'MVTOPPH0TGRY3NECS3K53CTS0V1IAFJRG4DPF1JGAIEBCO4W' # your Foursquare ID
CLIENT_SECRET = 'MN2AKKTPXWIKX13IUC1F0XHDXOVB1XHXMV5Q2CVTUW5RE1AB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

Define a function converting address to longitude and latitude. 

In [4]:
def address2location(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return location.longitude,location.latitude

Set current home address and future office address around which we would like to search for a new home location

In [5]:
homeAddress = '213 Rue Jean Jacques Rousseau, 92130 Issy-les-Moulineaux, France'
homeLongitude,homeLatitude = address2location(homeAddress)


Define Foursquare url to carry out explore. 

In [6]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, homeLatitude, homeLongitude, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef6780030567d545e80ec19'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Issy-les-Moulineaux',
  'headerFullLocation': 'Issy-les-Moulineaux, Paris',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 73,
  'suggestedBounds': {'ne': {'lat': 48.83141760900001,
    'lng': 2.271401296161224},
   'sw': {'lat': 48.81341759099999, 'lng': 2.244113103838776}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '541485c1498efe97bd3efa43',
       'name': 'La Passerelle',
       'location': {'address': '172 quai de Stalingrad',
        'lat': 48.82526662415111,
        'lng': 2.2575543895488503,
        'labeledLatLngs': [{'label': 'display',
 

## Data cleaning

First convert Foursquare results into pandas dataframe. 

In [7]:
# Define a function to extract category type 
def get_category_type(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    if len(category_list) ==0:
        return None
    else:
        return category_list[0]['name']
    
# define a function to merge formatted address into address string
def get_address(address_list):
    address=address_list[0]
    for ele in address_list[1:]:
        address = address + ', '+ele
    return address
        

Get venues and check columns for interested columns.

In [8]:
venues = results['response']['groups'][0]['items']
venues = json_normalize(venues)
venues.columns

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.venuePage.id', 'venue.location.crossStreet',
       'venue.location.neighborhood'],
      dtype='object')

Filter columns to get interested information, including name, category, address, longitude, latitude and distance to office address.

In [9]:
venues_filtered = venues.loc[:,['venue.categories','venue.location.distance','venue.location.formattedAddress','venue.location.lat','venue.location.lng','venue.name']]
venues_filtered.head()

,venue.categories,venue.location.distance,venue.location.formattedAddress,venue.location.lat,venue.location.lng,venue.name
0,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",317,"[172 quai de Stalingrad, 92130 Issy-les-Moulin...",48.825267,2.257554,La Passerelle
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",454,"[170 quai de Stalingrad, 92130 Issy-les-Moulin...",48.826249,2.255630,Île Saint-Germain
2,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",434,"[113 bis avenue de Verdun, 92130 Issy-les-Moul...",48.818860,2.255304,Issy Guinguette
3,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",374,"[1-6 boulevard Garibaldi, 92130 Issy-les-Mouli...",48.823489,2.262604,MurMur
4,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",317,"[20 Cours Saint Vincent, 92130 Issy-les-Moulin...",48.821011,2.253994,Le Cube


Apply functions *get_category_type*, *get_address* on corresponding columns and change colums to short names.

In [10]:
venues_filtered['venue.categories'] = venues_filtered.apply(get_category_type,axis=1)
venues_filtered['venue.location.formattedAddress'] =  venues_filtered['venue.location.formattedAddress'].map(get_address)

venues_filtered.columns = ['category','distance','address','latitude','longitude','name']
venues_filtered.head()

,category,distance,address,latitude,longitude,name
0,Modern European Restaurant,317,"172 quai de Stalingrad, 92130 Issy-les-Mouline...",48.825267,2.257554,La Passerelle
1,Park,454,"170 quai de Stalingrad, 92130 Issy-les-Mouline...",48.826249,2.255630,Île Saint-Germain
2,French Restaurant,434,"113 bis avenue de Verdun, 92130 Issy-les-Mouli...",48.818860,2.255304,Issy Guinguette
3,Gym / Fitness Center,374,"1-6 boulevard Garibaldi, 92130 Issy-les-Moulin...",48.823489,2.262604,MurMur
4,Art Museum,317,"20 Cours Saint Vincent, 92130 Issy-les-Mouline...",48.821011,2.253994,Le Cube


In [11]:
venues_filtered.sort_values(['distance'],ascending = True, inplace = True)
venues_filtered.tail()

,category,distance,address,latitude,longitude,name
31,Creperie,963,"10 square Louis Blériot, 92130 Issy-les-Moulin...",48.829034,2.266242,Crêperie Belle Isle Square
28,Brasserie,966,"26 bis rue hoche (rue danton), 92130 Issy-les-...",48.824426,2.270586,Le Zinc
37,Stadium,968,"6 boulevard des Frères Voisins, 92130 Issy-les...",48.827561,2.268412,Palais des Sports Robert Charpentier
21,Bakery,981,"Rue Camille Desmoulins, 92130 Issy-les-Mouline...",48.829874,2.264911,Maison Charpentier
19,Deli / Bodega,986,"27 rue Yves Kermen, 92100 Boulogne-Billancourt...",48.825911,2.245383,Bellota-Bellota Byzance


Class catogory type into four groups: 
1. **Commerce**,  including all types of restaurants, bakeries, creperie, pizza, Burger, coffee shops, tea houses, cocktail bars, brasseries, bistols, hotels, all shops, markets etc. 
2. **Culture**, including pub, museums, theaters, all sport centers, Music Venue, gallaries
3. **Transport**, train stations, 
5. **Environement**, gardens, parks and Fountain

Display all category types.

In [12]:
category_str=''
for i,ele in enumerate(venues_filtered['category'].unique()):
    category_str = category_str  +'{}. {} '.format(i+1,ele)
category_str

'1. Hotel 2. Burger Joint 3. Italian Restaurant 4. Japanese Restaurant 5. Supermarket 6. Boat or Ferry 7. Modern European Restaurant 8. Art Museum 9. Tram Station 10. Gym / Fitness Center 11. French Restaurant 12. Bistro 13. Park 14. Sandwich Place 15. Shopping Mall 16. Clothing Store 17. Bus Stop 18. Tennis Stadium 19. Playground 20. Falafel Restaurant 21. Event Service 22. Pizza Place 23. Food Truck 24. Café 25. Pool 26. Train Station 27. Restaurant 28. Bakery 29. Creperie 30. Brasserie 31. Stadium 32. Deli / Bodega '

Convert venue results into a feature vector where the distance element is normalized with respect to the search radius (set to 2 km here). 

In [13]:

def number2level(number_cat, levels, level_edges):
    edges_low=level_edges.copy()
    edges_low.insert(0,0)
    edges_high=level_edges.copy()
    edges_high.append(100)

    for level,elow,ehigh in zip(levels,edges_low,edges_high):
#         print(f'i= {i} number_cat ={number_cat} l_e[i-1]={elow} l_e[i]={ehigh} {(number_cat>elow) and (number_cat<=ehigh)}')
        if (number_cat>=elow) and (number_cat<ehigh): return level
# define a function to convert venue data into neighborhood feature array
def venue2feature(venue):
    commerce_keywords = ['Restaurant','Bakery','Salad','Café','Market','Supermarket','Pizza','Tea','Shop','Bar','Brasserie','Bistol','Hotel','Creperie','Burger']
    culture_keywords = ['Pub','Museum','Court','Sport','Gym','Fitness','Stadium','Rink','Music','Theatre','Gallary','Art','Studio','Spa','Pool','port','Port']
    transport_keywords = ['Train','Airport','Bus','Rail','RER','Tramway']
    environement_keywords = ['Garden','Park','Forest','Fountain','Forêt']
    
    
    dis_commerce = []
    cat_commerce = []
    dis_culture = []
    cat_culture  = []
    dis_transport = []
    cat_transport = []
    dis_environement = []
    cat_environement  = []
    venue['cato_commerce'] = venue.category.str.contains('|'.join(commerce_keywords)).astype(int)
    dis_commerce.append(venue['distance'][venue['cato_commerce']>0].mean() /radius)
    cat_commerce.append(number2level(venue['cato_commerce'].sum(),['low','median','high'],[5,20]))
    

    
    
    venue['cato_culture'] = venue.category.str.contains('|'.join(culture_keywords)).astype(int)
    dis_culture.append(venue['distance'][venue['cato_culture']>0].mean()/ radius)
    cat_culture.append(number2level(venue['cato_culture'].sum(),['low','median','high'],[5,20]))
    
    venue['cato_transport'] = venue.category.str.contains('|'.join(transport_keywords)).astype(int)
    dis_transport.append(venue['distance'][venue['cato_transport']>0].mean()/ radius)
    cat_transport.append(number2level(venue['cato_transport'].sum(),['none','few','many'],[1,3]))
    
    venue['cato_environement'] = venue.category.str.contains('|'.join(environement_keywords)).astype(int)
    dis_environement.append(venue['distance'][venue['cato_environement']>0].mean() / radius)
    cat_environement.append(number2level(venue['cato_environement'].sum(),['low','median','high'],[3,8]))
    
    feature=pd.DataFrame({'dis_commerce': dis_commerce, 'num_commerce': cat_commerce,
                          'dis_culture': dis_culture, 'num_culture': cat_culture,
                          'dis_transport': dis_transport, 'num_transport': cat_transport,
                          'dis_environement': dis_environement, 'num_environement': cat_environement,
                         })
    
    return venue,feature

venue, feature_home = venue2feature(venues_filtered)
feature_home

,dis_commerce,num_commerce,dis_culture,num_culture,dis_transport,num_transport,dis_environement,num_environement
0,0.67481,high,0.659,median,0.7702,many,0.5485,low


## Data preparation around office adresse in radius of 10 km

Segment the searching area into small regions of mesh grids. Each small region covers a radius of 2 km centered on the grids middles. In the area around the office, two locations with a diatance around 3 km have differences in longitude and latitude around 0.0150145 $^{\circ}$. 

In [14]:
officeAddress = '100 Avenue de Paris, 91344 Massy, France'
longitude,latitude = address2location(officeAddress)
print(f'office : longitude = {longitude} latitude={latitude}')
step_lat = 0.018
step_lon = 0.018
dist = mpu.haversine_distance((latitude,longitude), (latitude+step_lat,longitude))
dist

office : longitude = 2.2643366 latitude=48.723333


2.001508679602133

Segment 12 km around the office into 6 $\times$ 6 grids and store the latotudes and longitudes of grid centers in *location_list*.

In [15]:
longitude_step_list = [0.018]*6
latitude_step_list = [0.018]*6
location_list=[]
for i,step_lat in enumerate(latitude_step_list):
    for j,step_lon in enumerate(longitude_step_list):
#         print(f"i = {i-3} j={j-3} (longitude, latitude) = {(latitude+(2*i-5)*step_lat,longitude+(2*j-5)*step_lon)} ")
        dist = mpu.haversine_distance((latitude,longitude), (latitude+(2*i-5)*step_lat,longitude+(2*j-5)*step_lon))    
#         print(f"step_lat Distance = {dist} km")        
        location_list.append((latitude+(2*i-5)*step_lat,longitude+(2*j-5)*step_lon))
len(location_list)


36

Run the same data request, data cleaning for all the grids locations.

In [16]:
features=pd.DataFrame()
lat_list=[]
lon_list=[]
for (latitude,longitude) in location_list:
#     print(f"latitude={latitude}, longitude ={longitude}")
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    venues = json_normalize(venues)
    venues_filtered = venues.loc[:,['venue.categories','venue.location.distance','venue.location.formattedAddress','venue.location.lat','venue.location.lng','venue.name']]
    venues_filtered['venue.categories'] = venues_filtered.apply(get_category_type,axis=1)
    venues_filtered['venue.location.formattedAddress'] =  venues_filtered['venue.location.formattedAddress'].map(get_address)

    venues_filtered.columns = ['category','distance','address','latitude','longitude','name']
    venues_filtered.sort_values(['distance'],ascending = True, inplace = True)
    venue, feature = venue2feature(venues_filtered)
    features=features.append(feature)
    
    lat_list.append(latitude)
    lon_list.append(longitude)


features.head()


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,dis_commerce,num_commerce,dis_culture,num_culture,dis_transport,num_transport,dis_environement,num_environement
0,NaN,low,NaN,low,NaN,none,NaN,low
0,0.664000,low,NaN,low,NaN,none,NaN,low
0,0.881000,low,NaN,low,NaN,none,NaN,low
0,NaN,low,0.612,low,NaN,none,0.7685,low
0,0.504333,low,NaN,low,0.908,few,NaN,low


Merge home feature into district features for further one hot encoding

In [17]:
features=pd.concat([feature_home, features], ignore_index=True)
features.head()

,dis_commerce,num_commerce,dis_culture,num_culture,dis_transport,num_transport,dis_environement,num_environement
0,0.67481,high,0.659,median,0.7702,many,0.5485,low
1,NaN,low,NaN,low,NaN,none,NaN,low
2,0.66400,low,NaN,low,NaN,none,NaN,low
3,0.88100,low,NaN,low,NaN,none,NaN,low
4,NaN,low,0.612,low,NaN,none,0.7685,low


One hot encoding of feature elements *num_commerce*, *num_culture*, *num_transport* and *num_environement*

In [18]:
features_office = pd.get_dummies(data=features, columns=['num_commerce', 'num_culture','num_transport','num_environement'])
features_office.head()

,dis_commerce,dis_culture,dis_transport,dis_environement,num_commerce_high,num_commerce_low,num_commerce_median,num_culture_low,num_culture_median,num_transport_few,num_transport_many,num_transport_none,num_environement_low,num_environement_median
0,0.67481,0.659,0.7702,0.5485,1,0,0,0,1,0,1,0,1,0
1,NaN,NaN,NaN,NaN,0,1,0,1,0,0,0,1,1,0
2,0.66400,NaN,NaN,NaN,0,1,0,1,0,0,0,1,1,0
3,0.88100,NaN,NaN,NaN,0,1,0,1,0,0,0,1,1,0
4,NaN,0.612,NaN,0.7685,0,1,0,1,0,0,0,1,1,0


One problem with the data is that there are NaN values. For example, when it has no transport station located in the district of concern, the final feature value for *dis_transport* will be NaN. In order to fix this NaN Values, we replace NaN by 1 which indicates there no liocations are found in the district of concern

In [19]:
features_office.fillna(value=1,inplace=True)
features_office.head()


,dis_commerce,dis_culture,dis_transport,dis_environement,num_commerce_high,num_commerce_low,num_commerce_median,num_culture_low,num_culture_median,num_transport_few,num_transport_many,num_transport_none,num_environement_low,num_environement_median
0,0.67481,0.659,0.7702,0.5485,1,0,0,0,1,0,1,0,1,0
1,1.00000,1.000,1.0000,1.0000,0,1,0,1,0,0,0,1,1,0
2,0.66400,1.000,1.0000,1.0000,0,1,0,1,0,0,0,1,1,0
3,0.88100,1.000,1.0000,1.0000,0,1,0,1,0,0,0,1,1,0
4,1.00000,0.612,1.0000,0.7685,0,1,0,1,0,0,0,1,1,0


Now we have all the features in *features_office*. The first row is the current feature while the rest are the searched districts near future office. Now, we can calculate the similarity of the districts compared to current home district.  

In [20]:
feature_home = features_office.iloc[0]
features_office.drop([0],inplace= True)
features_office.head()

,dis_commerce,dis_culture,dis_transport,dis_environement,num_commerce_high,num_commerce_low,num_commerce_median,num_culture_low,num_culture_median,num_transport_few,num_transport_many,num_transport_none,num_environement_low,num_environement_median
1,1.000000,1.000,1.000,1.0000,0,1,0,1,0,0,0,1,1,0
2,0.664000,1.000,1.000,1.0000,0,1,0,1,0,0,0,1,1,0
3,0.881000,1.000,1.000,1.0000,0,1,0,1,0,0,0,1,1,0
4,1.000000,0.612,1.000,0.7685,0,1,0,1,0,0,0,1,1,0
5,0.504333,1.000,0.908,1.0000,0,1,0,1,0,1,0,0,1,0


Display home feature.

In [21]:
feature_home

dis_commerce               0.67481
dis_culture                0.65900
dis_transport              0.77020
dis_environement           0.54850
num_commerce_high          1.00000
num_commerce_low           0.00000
num_commerce_median        0.00000
num_culture_low            0.00000
num_culture_median         1.00000
num_transport_few          0.00000
num_transport_many         1.00000
num_transport_none         0.00000
num_environement_low       1.00000
num_environement_median    0.00000
Name: 0, dtype: float64

In [22]:
difference=features_office.sub(feature_home.iloc[0], axis = 'index')

difference.head()

,dis_commerce,dis_culture,dis_transport,dis_environement,num_commerce_high,num_commerce_low,num_commerce_median,num_culture_low,num_culture_median,num_transport_few,num_transport_many,num_transport_none,num_environement_low,num_environement_median
1,0.325190,0.32519,0.32519,0.32519,-0.67481,0.32519,-0.67481,0.32519,-0.67481,-0.67481,-0.67481,0.32519,0.32519,-0.67481
2,-0.010810,0.32519,0.32519,0.32519,-0.67481,0.32519,-0.67481,0.32519,-0.67481,-0.67481,-0.67481,0.32519,0.32519,-0.67481
3,0.206190,0.32519,0.32519,0.32519,-0.67481,0.32519,-0.67481,0.32519,-0.67481,-0.67481,-0.67481,0.32519,0.32519,-0.67481
4,0.325190,-0.06281,0.32519,0.09369,-0.67481,0.32519,-0.67481,0.32519,-0.67481,-0.67481,-0.67481,0.32519,0.32519,-0.67481
5,-0.170476,0.32519,0.23319,0.32519,-0.67481,0.32519,-0.67481,0.32519,-0.67481,0.32519,-0.67481,-0.67481,0.32519,-0.67481


In [23]:
square_error=pd.DataFrame(difference.pow(2).sum(axis=1))
square_error.columns=['square_error']
square_error['latitude'] = lat_list
square_error['longitude'] = lon_list
square_error.sort_values(by = ['square_error']).head(5)

,square_error,latitude,longitude
26,3.171730,48.777333,2.210337
35,3.203153,48.813333,2.318337
34,3.214943,48.813333,2.282337
33,3.234100,48.813333,2.246337
36,3.246096,48.813333,2.354337


## Results showed on map

Display the Top three homeFinder results on map. The large blue circle gives the searching radius while the three home icons are the top three locations for a new home location. The orange circles give the radius of my recommentations. 

In [28]:
longitude_office,latitude_office = address2location(officeAddress)
office_map = folium.Map(location=[latitude_office,longitude_office], zoom_start=10)
folium.Marker(
    location=[latitude_office,longitude_office],
    popup='Office',
    icon=folium.Icon(icon='office')
).add_to(office_map)
folium.Circle(
    location=[latitude_office,longitude_office],
    radius=12000,
    popup='Search radius',
    color='#3186cc',
    fill=False,
    fill_color='#3186cc',
    opacity =0.75
).add_to(office_map)

top_three = square_error.sort_values(by = ['square_error']).head(3)

for lat,lon in zip(top_three['latitude'],top_three['longitude']):
    folium.Circle(
        location=[lat,lon],
        radius=1000,
        popup='top 3',
        color='orange',
        fill=False,
        fill_color='moccasin',
    ).add_to(office_map)
    folium.Marker(
        location=[lat,lon],
        popup=f'latitude={lat}, longitude={lon}',
        icon=folium.Icon(icon='home')
    ).add_to(office_map)
office_map.save('HomeFinder.html')
# display map
office_map